In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from io import StringIO
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.exceptions import ResourceExistsError

# fill in .env file and read env vars
load_dotenv('.env')
CONNECTION_STRING = os.getenv('CONNECTION_STRING') # connection string of the storage account

In [2]:
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)

import pickle
with open("df_jsonls.pkl", "rb") as f:
    df_jsonls = pickle.load(f)

> replaced = ["COMPLETION_REPORT_1.PDF", "BIOSTRAT_REPORT_1.PDF"]

In [21]:
# container_client = blob_service_client.get_container_client("from-equinor")

# i = 0
# for blob in container_client.list_blobs(name_starts_with="raw"):
#     if blob.name.split("/")[-1] == "BIOSTRAT_REPORT_1.PDF":
#         blob_client = container_client.get_blob_client(blob)
#         os.makedirs("tmp", exist_ok=True)
#         with open(f"tmp/tmp-{str(i).zfill(2)}.pdf", "wb") as f:
#             f.write(blob_client.download_blob().readall())
#             i += 1
        

In [4]:
import base64
import urllib.parse
def decode_path(coded):
    # coded = coded[:-1] if coded[-1] == '0' else coded
    coded = coded[:-1] if coded[-1] in [str(e) for e in list(range(10))] else coded
    coded = coded + 4*"="
    coded = base64.b64decode(coded).decode('utf-8')
    return urllib.parse.unquote(coded)

In [18]:
container_client = blob_service_client.get_container_client('from-equinor-tmp')

blob_names = []
doc_names = []
for blob in container_client.list_blobs():
    name = decode_path(blob.name.split("/")[0])
    if name not in doc_names:
        blob_names.append(blob.name.split("/")[0])
        doc_names.append(name)

In [19]:
df = pd.DataFrame(list(zip(blob_names,doc_names)), columns=['blob_name', 'doc_name'])
df['doc_name'] = df['doc_name'].apply(lambda x: x.split('/')[-1])

df_ = df_jsonls[['doc_name']]
df_['page'] = df_['doc_name'].apply(lambda x: x.split('.')[-2].lstrip('0'))
df_['doc_name'] = df_['doc_name'].apply(lambda x: x.split('.0')[0])

df_ = df_.groupby(['doc_name']).aggregate(lambda x: x.tolist()).reset_index()

In [23]:
set_df = set(df['doc_name'].tolist())
set_df2 = set(df2['doc_name'].tolist())
set_df.difference(set_df2)

{'COMPLETION_REPORT_1.pdf'}

In [22]:
df2 = df_.merge(df, on='doc_name')
df2

,doc_name,page,blob_name
0,0030_09_01_CompletionReport.pdf,"[33, 42, 45]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
1,0031_03_02_Production test report_by Hydro.pdf,[7],aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
2,30-06-01 Biostratigraphical study of he cretac...,"[6, 8, 9]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
3,30-06-01 Biostratigraphical study of he cretac...,"[11, 12]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
4,"30-06-01 Biostratigraphy (220-3172,75).pdf","[2, 17]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
5,30-06-01 Biostratigraphy of the interval 2098-...,"[25, 13]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
6,30-06-01 Geological prognosis and drilling pro...,[58],aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
7,30-06-01_PB-706-0090.pdf,"[14, 54, 36]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
8,30-06-02 Paleontological Stratigraphical final...,"[9, 12]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...
9,30-06-02-Premilinary results.pdf,"[18, 6, 12]",aHR0cHM6Ly9rbW9nc3RvcmFnZS5ibG9iLmNvcmUud2luZG...


In [24]:
for i, row in df2.iterrows():
    for page in row['page']:
        blob_client = container_client.get_blob_client(row['blob_name'] + f'/normalized_images_{page}.jpg')
        fname = row['doc_name'] + f'/normalized_images_{page.zfill(4)}.jpg'
        os.makedirs(os.path.dirname("download_images/"+fname), exist_ok=True)
        with open(f"download_images/{fname}", "wb") as f:
            f.write(blob_client.download_blob().readall())
        

In [91]:
# container_client = blob_service_client.get_container_client("from-equinor")

# for d, _, fs, in os.walk('download_images/'):
#     for f in fs:
#         f_path = Path(d)/f
#         blob_path = f_path.relative_to('download_images')
#         blob_path = Path("NER/form-recognizer-images-4labeling")/blob_path
#         blob_client = blob_service_client.get_blob_client(container="from-equinor", blob=str(blob_path))
#         with open(f_path, 'rb') as data:
#             blob_client.upload_blob(data)
#         # # blob_path = os.path.join(d, f).split("/")